In [1]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from numpy import linalg
#from scipy import optimize
import matplotlib as mpl
import copy
from collections import defaultdict
from sklearn_extra.cluster import KMedoids
import math
import csv
# import matlab.engine

In [2]:
import random

In [48]:
X1 = [[x] for x in range(200)]
X2 = [[x] for x in range(250,450)]
X_adv = [[14] for _ in range(200)]
X = np.array(X1+X2)

S1 = [random.choice([0,1]) for x in range(200)]
S2 = [random.choice([0,1]) for x in range(200)]

S = np.array(S1+S2)

In [59]:
KM = KMedoids(2).fit(np.array(X1+X2+X_adv))

In [60]:
KM.cluster_centers_

array([[ 14],
       [337]])

In [61]:
KM.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [4]:
cluster_size = 2

In [5]:
from src.cluster import ClusteringKMedoid

In [10]:
final_clustering = ClusteringKMedoid("adversarial", cluster_size, X, S)
(
    centers_opt,
    adv_centers_idx,
    adv_members,
    _,
) = final_clustering.get_centers()

{1: 0.52, 0: 0.48}


In [11]:
centers_opt

array([[14],
       [ 1]])

In [12]:
####################
tmp_cntrs = []
for center in centers_opt:
    tmp_cntrs.append(np.sum(center))
tmp_cntrs = np.array(tmp_cntrs)
argsort_tmp_cntrs = np.argsort(tmp_cntrs)
final_cntrs = []
for i,_ in enumerate(centers_opt):
    final_cntrs.append(centers_opt[argsort_tmp_cntrs[i]])
final_cntrs = np.array(final_cntrs)
centers_opt = final_cntrs
####################

In [13]:
centers_opt

array([[ 1],
       [14]])

In [14]:
pr_groups=S

In [15]:
def fairness_cost_balance(data, lbls, groups):
    
    npoints = data.shape[0]
    num_groups = np.unique(groups).shape[0]

    ideal_proportion = defaultdict(float)
    for g in range(num_groups):
        ideal_proportion[g] = np.count_nonzero(np.array(groups == g))

    for g in range(num_groups):
        ideal_proportion[g] /= float(npoints)
    
    membership = defaultdict(lambda: defaultdict(float))
    cluster_sizes = defaultdict(float)
    for idx, x in enumerate(data):
        cluster_k = lbls[idx]
        for g in range(num_groups):
            if groups[idx] == g:
                membership[g][cluster_k] += 1.0
                 
        cluster_sizes[cluster_k] += 1.0

    val = float('inf')
    
    for cluster_k in np.unique(lbls):
        for g in range(num_groups):
            if(membership[g][cluster_k] == 0):
                return 0
            
            a = (float(membership[g][cluster_k])/float(cluster_sizes[cluster_k]))/float(ideal_proportion[g])
            b = float(ideal_proportion[g])/(float(membership[g][cluster_k])/float(cluster_sizes[cluster_k]))
            val = min(min(a, b), val)
    
    return val

In [16]:
plotting_on = 1

In [17]:
def run_kmedoids(n_clusters, X):
    #kmed = KMedoids(n_clusters = cluster_size, random_state=42).fit(X)
    # eng = matlab.engine.start_matlab()
    # matlab_X = matlab.double(X.tolist())
    # #matlab_labels, matlab_centers = eng.kmedoids(matlab_X, cluster_size, 'Distance', 'Euclidean' ,nargout = 2)
    # matlab_labels, matlab_centers = eng.kmedoids(matlab_X, cluster_size, nargout = 2)
    KM = KMedoids(n_clusters=2).fit(np.float64(X))
    matlab_labels, matlab_centers = KM.labels_, KM.cluster_centers_
    
    centers = []
    for cntr in matlab_centers:
        centers.append(np.array(cntr))
    centers = np.array(centers)
    
    
    tmp_cntrs = []
    for center in centers:
        tmp_cntrs.append(np.sum(center))
    tmp_cntrs = np.array(tmp_cntrs)
    argsort_tmp_cntrs = np.argsort(tmp_cntrs)
    final_cntrs = []
    for i,_ in enumerate(centers):
        final_cntrs.append(centers[argsort_tmp_cntrs[i]])
    final_cntrs = np.array(final_cntrs)
    centers = final_cntrs

    labels = tmp_kmeans(X, centers)
    
    return (labels, centers)

In [18]:
def plot_clusters(X, labels, centers):
    chosen_pts_cluster_label = []
    
    for idx, label in enumerate(labels):
        if idx in mask_idx:
            if plotting_on:
                plt.scatter(X[idx,0], X[idx,1], color = 'orange', marker = 's', s = 30)
                continue
        
        if label == 0:
            plt.scatter(X[idx,0], X[idx,1], color = 'lawngreen',s = 30)
            
        if label == 1:
            plt.scatter(X[idx,0], X[idx,1], color = 'red', s = 30)            
                
            
    if plotting_on:
        plt.scatter(centers[0,0], centers[0,1], color = 'blue', marker = '*', s = 30)
        
        plt.scatter(centers[1,0], centers[1,1], color = 'purple', marker = '*', s = 30)

    
    plt.show()

In [19]:
def tmp_kmeans(data, cntrs):
    tmp_labels = []
    for idx,x in enumerate(data):
        if cluster_size == 1:
            tmp_labels.append(0)
        if cluster_size == 2:
            tmp_labels.append(np.argmin([linalg.norm(cntrs[0] - x), linalg.norm(cntrs[1] - x)]))
        if cluster_size == 3:
            tmp_labels.append(np.argmin([linalg.norm(cntrs[0] - x), linalg.norm(cntrs[1] - x), linalg.norm(cntrs[2] - x)]))
        if cluster_size == 4:
            tmp_labels.append(np.argmin([linalg.norm(cntrs[0] - x), linalg.norm(cntrs[1] - x), linalg.norm(cntrs[2] - x), linalg.norm(cntrs[3] - x)]))
        if cluster_size == 5:
            tmp_labels.append(np.argmin([linalg.norm(cntrs[0] - x), linalg.norm(cntrs[1] - x), linalg.norm(cntrs[2] - x), linalg.norm(cntrs[3] - x), linalg.norm(cntrs[4] - x)]))
            
    
    return tmp_labels

In [20]:
mask_idx = []
# X = []
# with open('./datafiles/data_' + dataset_name + '.csv', newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=',')
#     for row in spamreader:
#         X.append(row)
# X = np.array(X).astype(np.float)
print(X.shape)
sample_size = X.shape[0]

(400, 1)


In [22]:
print(fairness_cost_balance(X, labels, pr_groups))

0.9687500000000001


In [23]:
labels_opt = tmp_kmeans(X, centers_opt)
print(fairness_cost_balance(X, labels_opt, pr_groups))

0.2604166666666667


In [24]:
print(linalg.norm(centers - centers_opt))

349.04011230802683


In [47]:
centers_opt

array([[ 1],
       [14]])

In [121]:
from itertools import permutations

In [122]:
centers_opt

array([[ 1],
       [14]])

In [137]:
X1 = [[x] for x in range(200)]
X2 = [[x] for x in range(250,450)]
X = np.array(X1+X2)

S1 = [random.choice([0,1]) for x in range(200)]
S2 = [random.choice([0,1]) for x in range(200)]

S = np.array(S1+S2)

labels, centers = run_kmedoids(cluster_size, X)
print(centers)


V_k = [0]*cluster_size
ep_k = [20]*cluster_size
test=0
print(centers_opt)
co_perms = [centers_opt[[i]] for i in perms]
perms = []
for p in permutations(range(cluster_size)):
    perms.append(list(p))
while np.linalg.norm(centers - centers_opt) != 0:
    if np.linalg.norm(centers - centers_opt[::-1]) == 0:
        break

# while test<100:
#     test+=1
    if cluster_size >= 2:
        done=False
        c_perms = [centers[i] for i in perms]
        for c in c_perms:
            for co in co_perms:
                if np.linalg.norm(c - co) == 0:
                    done = True
                    break
        if done:
            break
    ######## This code segment checks to see if we have arrived at similar centers since they might be shuffled (clustering is never consistent in labeling) #########################
    #################################
    for k_i in range(cluster_size):
        print("os", k_i, centers[k_i], centers_opt[k_i])
        if np.linalg.norm(centers[k_i] - centers_opt[k_i]) != 0:
            print("is", k_i, centers[k_i], centers_opt[k_i])
            V = datasets.make_blobs(n_samples=ep_k[k_i], cluster_std=[0.0], random_state=170, centers = [centers_opt[k_i]])[0] #RS = 42000
            X = np.vstack((X, V))
            for i in range(sample_size, sample_size+ep_k[k_i]):
                mask_idx.append(i)
            sample_size += ep_k[k_i]
            V_k[k_i] += ep_k[k_i]
        labels, centers = run_kmedoids(cluster_size, X)
        # print("new_centers", centers)
        if np.linalg.norm(centers - centers_opt) == 0:
            break

    print(np.linalg.norm(centers-centers_opt))

[[ 99.]
 [349.]]
[[ 1]
 [14]]
os 0 [99.] [1]
is 0 [99.] [1]
os 1 [349.] [14]
is 1 [349.] [14]
343.96075357517174
os 0 [79.] [1]
is 0 [79.] [1]
os 1 [349.] [14]
is 1 [349.] [14]
339.98382314457257
os 0 [59.] [1]
is 0 [59.] [1]
os 1 [349.] [14]
is 1 [349.] [14]
331.6458954969894
os 0 [34.] [1]
is 0 [34.] [1]
os 1 [339.] [14]
is 1 [339.] [14]
323.2615040489665
os 0 [14.] [1]
is 0 [14.] [1]
os 1 [337.] [14]
is 1 [337.] [14]
323.2615040489665
os 0 [14.] [1]
is 0 [14.] [1]


/home/pranjal/venv/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


os 1 [337.] [14]
is 1 [337.] [14]
323.2615040489665
os 0 [14.] [1]
is 0 [14.] [1]
os 1 [337.] [14]
is 1 [337.] [14]
323.2615040489665
os 0 [14.] [1]
is 0 [14.] [1]
os 1 [337.] [14]
is 1 [337.] [14]
323.2615040489665
os 0 [14.] [1]
is 0 [14.] [1]
os 1 [337.] [14]
is 1 [337.] [14]
323.2615040489665
os 0 [14.] [1]
is 0 [14.] [1]
os 1 [14.] [14]
13.0
os 0 [14.] [1]
is 0 [14.] [1]


/home/pranjal/venv/lib/python3.6/site-packages/sklearn_extra/cluster/_k_medoids.py:279: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  "its corresponding cluster ({k}).".format(k=k)
/home/pranjal/venv/lib/python3.6/site-packages/sklearn_extra/cluster/_k_medoids.py:279: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  "its corresponding cluster ({k}).".format(k=k)
/home/pranjal/venv/lib/python3.6/site-packages/sklearn_extra/cluster/_k_medoids.py:279: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  "its corresponding cluster ({k}).".format(k=k)
/home/pranjal/venv/lib/python3.6/site-packages/sklearn_extra/cluster/_k_medoids.py:279: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1

os 1 [14.] [14]
13.0
os 0 [14.] [1]
is 0 [14.] [1]
os 1 [14.] [14]
13.0
os 0 [14.] [1]
is 0 [14.] [1]
os 1 [14.] [14]
13.0
os 0 [14.] [1]
is 0 [14.] [1]
os 1 [14.] [14]
13.0
os 0 [14.] [1]
is 0 [14.] [1]


...

os 1 [333.] [14]
is 1 [333.] [14]
13.0
os 0 [14.] [1]
is 0 [14.] [1]
os 1 [333.] [14]
is 1 [333.] [14]


/home/pranjal/venv/lib/python3.6/site-packages/sklearn_extra/cluster/_k_medoids.py:279: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  "its corresponding cluster ({k}).".format(k=k)
/home/pranjal/venv/lib/python3.6/site-packages/sklearn_extra/cluster/_k_medoids.py:279: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  "its corresponding cluster ({k}).".format(k=k)


13.0
os 0 [14.] [1]
is 0 [14.] [1]
os 1 [333.] [14]
is 1 [333.] [14]
13.0
os 0 [14.] [1]
is 0 [14.] [1]


/home/pranjal/venv/lib/python3.6/site-packages/sklearn_extra/cluster/_k_medoids.py:279: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  "its corresponding cluster ({k}).".format(k=k)
/home/pranjal/venv/lib/python3.6/site-packages/sklearn_extra/cluster/_k_medoids.py:279: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  "its corresponding cluster ({k}).".format(k=k)


os 1 [26.] [14]
is 1 [26.] [14]
13.0
os 0 [14.] [1]
is 0 [14.] [1]
0.0


In [138]:
print(X.shape)

(1540, 1)


In [139]:
np.unique(X[400:], axis=0,return_counts=True)

(array([[ 1.],
        [14.]]),
 array([760, 380]))

In [140]:
print(V_k)

[760, 380]


In [141]:
print(centers)
print(centers_opt)

[[ 1.]
 [14.]]
[[ 1]
 [14]]


In [22]:
if dataset_name == 'bankm': #'sanity_check'
    labels = tmp_kmeans(X, centers)
    print(fairness_cost_balance(X[:45211], labels, pr_groups))
    
    labels, centers = run_kmedoids(cluster_size, X)
    print(fairness_cost_balance(X[:45211], labels, pr_groups))
    
    
if dataset_name == 'creditcard': #'sanity_check'
    labels = tmp_kmeans(X, centers)
    print(fairness_cost_balance(X[:30000], labels, pr_groups))
    
    labels, centers = run_kmedoids(cluster_size, X)
    print(fairness_cost_balance(X[:30000], labels, pr_groups))


0
0


In [21]:
#BANKM: 45211, #ADULTM: 10000, #CREDITCARD: 30000

## k = 2
#BANKM: 100% decrease (100 [100, 0] points added) | 0.30374129684872203 -> 0.0
#CREDITCARD: 100% decrease (200 [100,100] points added) | 0.8611658242781424 -> 0.0
#ADULTM: 100% decrease (600 [500, 100] points added) | 0.5668357597268627 -> 0.0

## k = 3
#BANKM: 100% decrease (1300 [100, 600, 600] points added) | 0.29779400214116925 -> 0.0
#CREDITCARD: 100% decrease (300 [100, 100, 100] points added) | 0.7579775571824116 -> 0.0
#ADULTM: NA | 0.0 -> 0.0

## k = 4
#BANKM: 100% decrease (1900 [100, 200, 700, 900] points added) | 0.28292193008885463 -> 0.0
#CREDITCARD: 100% decrease (900 [400, 300, 100, 100] points added) | 0.4640690543560836 -> 0.0
#ADULTM: NA | 0.0 -> 0.0